## Now, let us load the required data into individual variables

In [4]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy import stats
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

np.random.seed(1)

In [5]:
X_train = pd.read_csv('C:/Users/suman/DspData/data/churn_train_X.csv') 
y_train = pd.read_csv('C:/Users/suman/DspData/data/churn_train_y.csv') 
X_test = pd.read_csv('C:/Users/suman/DspData/data/churn_test_X.csv') 
y_test = pd.read_csv('C:/Users/suman/DspData/data/churn_test_y.csv') 


Let us create a data frame called performance and put all the needed metrics in it!

In [6]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

#  Logistic Regression with Random Search

In [48]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'penalty': ['None','l1','l2','elasticnet'],
    'solver':['saga','liblinear']
}

model = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

c:\Users\suman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits
The best recall score is 0.5669050216135417
... with parameters: {'solver': 'liblinear', 'penalty': 'l1'}


c:\Users\suman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\suman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\suman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\suman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 441, in _check_solver
    raise ValueError(
Valu

#  Logistic Regression with Grid Search

In [49]:
score_measure = "recall"
kfolds = 5
penalty = rand_search.best_params_['penalty']
solver = rand_search.best_params_['solver']

param_grid = {  
    'penalty': [penalty],
    'solver': [solver]
}

model = LogisticRegression()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallLogistic = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\suman\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best recall score is 0.5695996444875369
... with parameters: {'penalty': 'l1', 'solver': 'liblinear'}


c:\Users\suman\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [50]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression",
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.903587,0.799458,0.57393,0.668177


## Decision Trees - Random Search 

In [52]:
from sklearn.tree import DecisionTreeClassifier
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,100),  
    'min_samples_leaf': np.arange(1,75),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 80), 
    'max_depth': np.arange(1,40), 
    'criterion': ['entropy', 'gini'],
}

model = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.7861754130812427
... with parameters: {'min_samples_split': 19, 'min_samples_leaf': 15, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 66, 'max_depth': 11, 'criterion': 'entropy'}


## DECISION TREE - GRID SEARCH

In [53]:
score_measure = "recall"
kfolds = 5
min_samples_split = rand_search.best_params_['min_samples_split']
min_samples_leaf = rand_search.best_params_['min_samples_leaf']
min_impurity_decrease = rand_search.best_params_['min_impurity_decrease']
max_leaf_nodes = rand_search.best_params_['max_leaf_nodes']
max_depth = rand_search.best_params_['max_depth']
criterion = rand_search.best_params_['criterion']

param_grid = {
    'min_samples_split': np.arange(min_samples_split-2,min_samples_split+2),  
    'min_samples_leaf': np.arange(min_samples_leaf-2,min_samples_leaf+2),
    'min_impurity_decrease': np.arange(min_impurity_decrease-0.0001, min_impurity_decrease+0.0001, 0.00005),
    'max_leaf_nodes': np.arange(max_leaf_nodes-2,max_leaf_nodes+2), 
    'max_depth': np.arange(max_depth-2,max_depth+2), 
    'criterion': [criterion]
}

model = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
The best recall score is 0.7969862238920535
... with parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_leaf_nodes': 65, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 13, 'min_samples_split': 17}


In [54]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.903587,0.799458,0.573930,0.668177
0,Decision Tree,0.940112,0.812030,0.840467,0.826004


## RandomForestClassifier

In [55]:
# Transform the dataset (only training data)
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_up, y_up = oversample.fit_resample(X_train, y_train)

In [56]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_up, y_up)

rfpred = rf.predict(X_test)

C:\Users\suman\AppData\Local\Temp\ipykernel_7920\1298015926.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_up, y_up)


In [57]:
c_matrix = (confusion_matrix(y_test, rfpred))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random Forest Classifier", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.903587,0.799458,0.573930,0.668177
0,Decision Tree,0.940112,0.812030,0.840467,0.826004
0,Random Forest Classifier,0.945048,0.801739,0.896887,0.846648


It's important that we don't predict churning as non-churning customers. That's why the model needs to be evaluated on the "Recall"- metric

Conclusion :The goal of this project is to provide an analysis which shows the difference between a non-churning and churning customer. Using the existing data managed to train a model with upsampled data which reaches a recall score of 89%.This will provide us insight into which customers are eager to churn.

# MLP Classifier

In [29]:

import tensorflow as tf
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_s, X_test_s = scaler.transform(X_train), scaler.transform(X_test)

In [30]:
%%time

model1 = MLPClassifier(
    hidden_layer_sizes=(60,50,40), 
    activation = 'relu',
    solver='adam',
    alpha=0.0001, # Strength of the L2 regularization term
    batch_size='auto',
    learning_rate = 'constant',
    learning_rate_init = 0.001,
    max_iter=200,
    tol=0.00001, 
    early_stopping = True,
    n_iter_no_change = 5,
    verbose=True
    
)
_ = model1.fit(X_train_s, y_train)

# Currently (version 1.2.2), MLPClassifier supports only the Cross-Entropy loss function.
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

c:\Users\suman\anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.48060017
Validation score: 0.839210
Iteration 2, loss = 0.32235005
Validation score: 0.895628
Iteration 3, loss = 0.26217806
Validation score: 0.906911
Iteration 4, loss = 0.23271762
Validation score: 0.909732
Iteration 5, loss = 0.21407224
Validation score: 0.916784
Iteration 6, loss = 0.19722730
Validation score: 0.915374
Iteration 7, loss = 0.18604824
Validation score: 0.908322
Iteration 8, loss = 0.17569567
Validation score: 0.913963
Iteration 9, loss = 0.16658780
Validation score: 0.913963
Iteration 10, loss = 0.15869492
Validation score: 0.918195
Iteration 11, loss = 0.15165963
Validation score: 0.911142
Iteration 12, loss = 0.14473661
Validation score: 0.918195
Iteration 13, loss = 0.14001538
Validation score: 0.913963
Iteration 14, loss = 0.13655168
Validation score: 0.919605
Iteration 15, loss = 0.13147575
Validation score: 0.919605
Iteration 16, loss = 0.12730806
Validation score: 0.919605
Iteration 17, loss = 0.12367332
Validation score: 0.919605
Iterat

In [31]:
model1.loss_curve_

[0.48060016857014554,
 0.3223500531666497,
 0.26217805916973924,
 0.2327176194594692,
 0.21407224239898323,
 0.1972273003284454,
 0.1860482361406369,
 0.17569566968572922,
 0.16658779540564875,
 0.1586949231605642,
 0.151659628427455,
 0.1447366144158229,
 0.1400153798368658,
 0.13655168484642002,
 0.1314757510797693,
 0.12730805997759273,
 0.1236733166059745,
 0.12072509396499922,
 0.11721138250124942,
 0.11704004340544966,
 0.11332071898770292,
 0.10951701609491211,
 0.10452151968631554,
 0.10002417364348773,
 0.09730656193186404,
 0.09591101940647796,
 0.0914864119538316,
 0.08956362456300648,
 0.08743704922169881,
 0.0842320329085246,
 0.08113693493822996,
 0.07737037774573213,
 0.07531973764668895,
 0.07320292471292175,
 0.07321800703076395,
 0.06831125181816657]

In [32]:
%%time
y_pred = model1.predict(X_test_s)

# from sklearn.metrics import f1_score, precision_score, recall_score
# from sklearn.metrics import confusion_matrix

# import matplotlib.pylab as plt

# y_train_pred, y_test_pred = (my_model.predict(X_train_s).flatten() > 0.5)*1, (my_model.predict(X_test_s).flatten() > 0.5) * 1

CPU times: total: 0 ns
Wall time: 8.37 ms


In [33]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9553    0.9490    0.9521      2547
           1     0.7446    0.7703    0.7572       492

    accuracy                         0.9200      3039
   macro avg     0.8500    0.8596    0.8547      3039
weighted avg     0.9212    0.9200    0.9206      3039



In [34]:
%%time

param_distributions = {
    'hidden_layer_sizes': [ (64,), (128,),(128,64), (64,128), (64,128,196), (196,128,64)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .0001, .0005, .001, .005],
    'batch_size': [25, 50, 100],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.0005, 0.001, 0.005, 0.01],
    'max_iter': [5000],
    'tol': [0.000005, 0.00001, 0.00005],
    'early_stopping':[True],
    'n_iter_no_change':[5],
}

random_search = RandomizedSearchCV(
    estimator = MLPClassifier(), # a blank slate... RandomizedSearchCV will send parameters.
    param_distributions=param_distributions, 
    cv=3, 
    n_iter=300,
    scoring='accuracy', # note that we could also choose any other scoring metric that is appropriate for a multi-class problem - such as f1_macro, f1_micro, f1_weighted, etc.
    verbose=1, 
    n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
    return_train_score=True
)

_ = random_search.fit(X_train_s, y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


c:\Users\suman\anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: total: 15.3 s
Wall time: 10min 20s


In [35]:
model2 = random_search.best_estimator_

print(random_search.best_params_)

{'tol': 5e-06, 'solver': 'adam', 'n_iter_no_change': 5, 'max_iter': 5000, 'learning_rate_init': 0.005, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (128, 64), 'early_stopping': True, 'batch_size': 25, 'alpha': 0.0005, 'activation': 'tanh'}


In [36]:
%%time
y_pred = model2.predict(X_test_s)

print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9428    0.9768    0.9595      2547
           1     0.8525    0.6931    0.7646       492

    accuracy                         0.9309      3039
   macro avg     0.8976    0.8350    0.8620      3039
weighted avg     0.9282    0.9309    0.9279      3039

CPU times: total: 62.5 ms
Wall time: 43.9 ms


# Keras

In [7]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [8]:
%%time

def build_clf(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    target_encoder_ = meta["target_encoder_"]
    
    model = tf.keras.models.Sequential()
    model.add(keras.layers.Input(shape=n_features_in_)),
    #for hidden_layer_size in hidden_layer_sizes:
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, 
            kernel_initializer= tf.keras.initializers.GlorotUniform(), 
            bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), 
            activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    #though you could return a compiled model, it's not necessary, and would result in the loss of these
    # parameters in the tune process - as they would be 'hard coded'
    # model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) 

    return model

CPU times: total: 0 ns
Wall time: 0 ns


In [9]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=64,
    dropout=0.5,
    optimizer=keras.optimizers.Adam,
    optimizer__learning_rate=0.0001
)
keras_clf.get_params()

CPU times: total: 0 ns
Wall time: 0 ns


{'model': <function __main__.build_clf(meta, hidden_layer_sizes, dropout)>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': keras.optimizers.optimizer_v2.adam.Adam,
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 1,
 'hidden_layer_sizes': 64,
 'dropout': 0.5,
 'optimizer__learning_rate': 0.0001,
 'class_weight': None}

In [10]:
%%time

params = {
    
    # the following are model parameters, and therefore must be defined as parameters in the KarasClassifier, and then in the build_clf function
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)], # this will require KarasClassifier and build_clf to have hidden_layer_sizes parameter set
    'model__dropout': [0, 0.1], # this will require KarasClassifier and build_clf to have hidden_layer_sizes parameter set
    
    # the following are 'fit' parameters, the scikeras wrapper provides these parameters. These are passed to the 'model.fit' method for each fit of the model
    'batch_size':[20, 60, 100],
    'epochs':[10],
    'optimizer':['adam','sgd'],
    'loss':['sparse_categorical_crossentropy'],
    
    # this is added to the optimizer 
    'optimizer__learning_rate': [0.0001, 0.001, 0.01]

}
keras_clf.get_params()

CPU times: total: 0 ns
Wall time: 0 ns


{'model': <function __main__.build_clf(meta, hidden_layer_sizes, dropout)>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': keras.optimizers.optimizer_v2.adam.Adam,
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 1,
 'hidden_layer_sizes': 64,
 'dropout': 0.5,
 'optimizer__learning_rate': 0.0001,
 'class_weight': None}

In [11]:
%%time

from sklearn.model_selection import RandomizedSearchCV
#from tensorflow.keras.callbacks import EarlyStopping

rnd_search_cv = RandomizedSearchCV(
    estimator=keras_clf, 
    param_distributions=params, 
    scoring='accuracy',  # we could use any appropriate sklearn metric here (i.e. accuracy, f1_micro, f1_macro)
    n_iter=50, 
    cv=3)

# In rare cases, you may find your model training results in exceeding python's default recursion limit.
# If needed, you can increase this excersion limit by using the following code.
#import sys
#sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

_ = rnd_search_cv.fit(X_train, y_train,  verbose=1)

# You can create 'call back' functions. These are functions that will be called at the 
# end of each epoch. There are a number of builtin functions created for this purpose, 
# one of which is EarlyStopping -- that, based on the parameters you give, will stop
# the training process. This is useful when the algorithm is not making any significant
# gains through further training. 
#earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
#callback = [earlystop]
#_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)

Epoch 1/10
79/79 [==============================] - 1s 5ms/step - loss: 72296689438457593856.0000
Epoch 2/10
79/79 [==============================] - 0s 3ms/step - loss: 0.9020
Epoch 3/10
79/79 [==============================] - 0s 4ms/step - loss: 0.9006
Epoch 4/10
79/79 [==============================] - 0s 4ms/step - loss: 0.9196
Epoch 5/10
79/79 [==============================] - 0s 4ms/step - loss: 0.9008
Epoch 6/10
79/79 [==============================] - 0s 4ms/step - loss: 0.8933
Epoch 7/10
79/79 [==============================] - 0s 3ms/step - loss: 0.9176
Epoch 8/10
79/79 [==============================] - 0s 4ms/step - loss: 0.8971
Epoch 9/10
79/79 [==============================] - 0s 4ms/step - loss: 0.8993
Epoch 10/10
40/40 [==============================] - 0s 3ms/step
Epoch 1/10
79/79 [==============================] - 1s 4ms/step - loss: 3068782903912764014592.0000
Epoch 2/10
79/79 [==============================] - 0s 4ms/step - loss: 2.2369
Epoch 3/10
79/79 [========

c:\Users\suman\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\suman\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\suman\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "c:\Users\suman\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\suman\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\suman\anaconda3\envs\tf\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.

Epoch 1/10
119/119 [==============================] - 1s 4ms/step - loss: 11434167701832722284544.0000
Epoch 2/10
119/119 [==============================] - 0s 4ms/step - loss: 0.9163
Epoch 3/10
119/119 [==============================] - 0s 4ms/step - loss: 0.9170
Epoch 4/10
119/119 [==============================] - 0s 4ms/step - loss: 0.9184
Epoch 5/10
119/119 [==============================] - 1s 4ms/step - loss: 0.9069
Epoch 6/10
119/119 [==============================] - 0s 4ms/step - loss: 0.8999
Epoch 7/10
119/119 [==============================] - 0s 4ms/step - loss: 0.9188
Epoch 8/10
119/119 [==============================] - 0s 4ms/step - loss: 0.9056
Epoch 9/10
119/119 [==============================] - 0s 4ms/step - loss: 0.8843
Epoch 10/10
119/119 [==============================] - 0s 4ms/step - loss: 0.8760
CPU times: total: 1h 21min 39s
Wall time: 12min 49s


In [12]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.001,
 'optimizer': 'sgd',
 'model__hidden_layer_sizes': (100, 90),
 'model__dropout': 0.1,
 'loss': 'sparse_categorical_crossentropy',
 'epochs': 10,
 'batch_size': 60}

In [13]:
best_model = rnd_search_cv.best_estimator_

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, best_model.predict(X_test_s), digits=3))

51/51 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

           0      0.830     0.675     0.745      2547
           1      0.146     0.287     0.193       492

    accuracy                          0.612      3039
   macro avg      0.488     0.481     0.469      3039
weighted avg      0.720     0.612     0.655      3039



### MLP Classifier and Keras Summary

It's important that we don't predict churning as non-churning customers. That's why the model needs to be evaluated on the "Recall"- metric

* Recall values all alogorithms
* MLP Classifier = 77%
* MLP classifier with random search = 69%
* keras with random search = 28%
* Logistic Regression =57%
* Decision tree = 84%
* Random Forest Classifier = 89%


Random Forest is less computationally expensive and does not require a GPU to finish training. A random forest can give you a different interpretation of a decision tree but with better performance. Neural Networks will require much more data than an everyday person might have on hand to actually be effective. The neural network will simply decimate the interpretability of your features to the point where it becomes meaningless for the sake of performance. While that may sound reasonable to some, it is dependent on each project.